# Model-1 
GraphSage + content embeddings using default parameters 3 Layer MLP.

In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-+.html

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Looking in links: https://data.pyg.org/whl/torch-+.html

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from torch_geometric.datasets import UPFD #importing the UPFD Dataset


In [21]:
#defining the train and test split by defining the feature as content and setting the name as politifact
test_data_pol = UPFD(root=".", name="politifact", feature= "content",split="test")
train_data_pol = UPFD(root=".", name="politifact", feature= "content", split="train")
val_data_pol = UPFD(root=".", name="politifact", feature= "content", split="val")
train_data_pol = train_data_pol + val_data_pol


In [22]:
print("Politifact Dataset")
print("Train Samples: ", len(train_data_pol))
print("Test Samples: ", len(test_data_pol))

Politifact Dataset
Train Samples:  93
Test Samples:  221


In [10]:
train_data_pol[0].edge_index


tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  8,  8, 16, 16, 16, 16, 16, 16,
         24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
         24, 24, 24, 24, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 60],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
         55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]])

In [11]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data_pol, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data_pol, batch_size=256, shuffle=False)

In [12]:
import torch
import torch.nn.functional as F
from torch.nn import LeakyReLU, Softmax, Linear, SELU,Dropout
from torch_geometric.nn import SAGEConv, global_max_pool, GATv2Conv, TopKPooling, global_mean_pool
from torch_geometric.transforms import ToUndirected
from torch.nn import LeakyReLU

In [13]:
#defining the GraphSage Model with 3 SageConv layers and 3 unit MLP
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        
        self.full1 = Linear(hidden_channels[2],hidden_channels[3])
        self.full2 = Linear(hidden_channels[3],hidden_channels[4])
        self.full3 = Linear(hidden_channels[4],hidden_channels[5])

        self.softmax = Linear(hidden_channels[5],out_channels)

        #droupouts
        self.dp1 = Dropout(0.2)
        self.dp2 = Dropout(0.2)
        self.dp3 = Dropout(0.2)

    def forward(self, x, edge_index, batch):
        h = self.conv1(x, edge_index).relu()
        h = self.conv2(h, edge_index).relu()
        h = self.conv3(h, edge_index).relu()

        h = global_max_pool(h,batch)

        h = self.full1(h).relu()
        h = self.dp1(h)
        h = self.full2(h).relu()
        h = self.dp2(h)
        h = self.full3(h).relu()
        h = self.dp3(h)
        
        h = self.softmax(h)

        return torch.sigmoid(h)

In [14]:
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, f1_score 

In [15]:
#specifying number of input features, hidden layer sizes, and number of output channels
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(test_data_pol.num_features,[512,512,512,256,256,256],1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
lossff = torch.nn.BCELoss()
print(device)

cpu


In [16]:
#defining the train and test function for the model
def train(epoch):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        # print(out)

        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test(epoch):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    # print(preds.cpu().numpy())

    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1


In [17]:
#printing out the epoch at lowest wloss. 
wloss = []
weighted_loss = 0
exp_param = 0.8
best_test_loss = float('inf')  # initialize with a large value

#without dropout training results
for epoch in range(500):
  train_loss = train(epoch)
  test_loss, test_acc, test_f1 = test(epoch)
  weighted_loss = exp_param * (weighted_loss) + (1 - exp_param) * (test_loss / len(test_loader.dataset))
  
  wloss.append(weighted_loss / (1 - exp_param ** (epoch + 1)))
  
  if test_loss < best_test_loss:
    best_test_loss = test_loss  # Update the best test loss

  print(f'Epoch: {epoch:02d} |  TrainLoss: {train_loss:.5f} | '
        f'TestLoss: {test_loss:.5f} | TestAcc: {test_acc:.5f} | TestF1: {test_f1:.2f}')

#print the best values
best_wloss = min(wloss)
best_epoch = wloss.index(best_wloss)
print(f'Best WLoss: {best_wloss:.5f} | Best Epoch: {best_epoch}')

Epoch: 00 |  TrainLoss: 0.69312 | TestLoss: 0.69312 | TestAcc: 0.49774 | TestF1: 0.43
Epoch: 01 |  TrainLoss: 0.69356 | TestLoss: 0.69319 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 02 |  TrainLoss: 0.69373 | TestLoss: 0.69328 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 03 |  TrainLoss: 0.69266 | TestLoss: 0.69334 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 04 |  TrainLoss: 0.69371 | TestLoss: 0.69336 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 05 |  TrainLoss: 0.69290 | TestLoss: 0.69340 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 06 |  TrainLoss: 0.69266 | TestLoss: 0.69344 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 07 |  TrainLoss: 0.69281 | TestLoss: 0.69348 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 08 |  TrainLoss: 0.69147 | TestLoss: 0.69350 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 09 |  TrainLoss: 0.69199 | TestLoss: 0.69352 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 10 |  TrainLoss: 0.69187 | TestLoss: 0.69356 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 11 |  TrainLoss: 0.69214 | TestLoss: 0.69359 | 

# Model 2: 

GraphSage + Content with hyperparameters as defined by the paper and 3 Layer MLP

(Embedding size = 128, batch size= 128, l2 Regularization = 0.001) 

In [23]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, embedding_size, batch_size, l2_reg_weight):
        super(Net, self).__init__()
        self.embedding_size = embedding_size
        self.batch_size = batch_size
        self.l2_reg_weight = l2_reg_weight
        
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        
        self.full1 = Linear(hidden_channels[2], hidden_channels[3])
        self.full2 = Linear(hidden_channels[3], hidden_channels[4])
        self.full3 = Linear(hidden_channels[4], hidden_channels[5])

        self.softmax = Linear(hidden_channels[5], out_channels)

        # Dropouts
        self.dp1 = Dropout(0.2)
        self.dp2 = Dropout(0.2)
        self.dp3 = Dropout(0.2)

    def forward(self, x, edge_index, batch):
        h = self.conv1(x, edge_index).relu()
        h = self.conv2(h, edge_index).relu()
        h = self.conv3(h, edge_index).relu()

        h = global_max_pool(h, batch)

        h = self.full1(h).relu()
        h = self.dp1(h)
        h = self.full2(h).relu()
        h = self.dp2(h)
        h = self.full3(h).relu()
        h = self.dp3(h)
        
        h = self.softmax(h)

        return torch.sigmoid(h)

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(test_data_pol.num_features,[512,512,512,256,256,256],1, 128, 128, 0.001).to(device) #setting embedding size=128, batch size=128, l2 regularization weight=0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
lossff = torch.nn.BCELoss()
print(device)

cpu


In [25]:
def train(epoch):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        # print(out)

        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test(epoch):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    # print(preds.cpu().numpy())

    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1

In [26]:
wloss = []
weighted_loss = 0
exp_param = 0.8
best_test_loss = float('inf')  # Initialize with a large value

# Without dropout training results
for epoch in range(500):
  train_loss = train(epoch)
  test_loss, test_acc, test_f1 = test(epoch)
  weighted_loss = exp_param * (weighted_loss) + (1 - exp_param) * (test_loss / len(test_loader.dataset))
  
  wloss.append(weighted_loss / (1 - exp_param ** (epoch + 1)))
  
  if test_loss < best_test_loss:
    best_test_loss = test_loss  # Update the best test loss

  print(f'Epoch: {epoch:02d} |  TrainLoss: {train_loss:.5f} | '
        f'TestLoss: {test_loss:.5f} | TestAcc: {test_acc:.5f} | TestF1: {test_f1:.2f}')

# Print the best values
best_wloss = min(wloss)
best_epoch = wloss.index(best_wloss)
print(f'Best WLoss: {best_wloss:.5f} | Best Epoch: {best_epoch}')

Epoch: 00 |  TrainLoss: 0.69222 | TestLoss: 0.69389 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 01 |  TrainLoss: 0.69244 | TestLoss: 0.69387 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 02 |  TrainLoss: 0.69245 | TestLoss: 0.69385 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 03 |  TrainLoss: 0.69243 | TestLoss: 0.69387 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 04 |  TrainLoss: 0.69078 | TestLoss: 0.69394 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 05 |  TrainLoss: 0.69128 | TestLoss: 0.69401 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 06 |  TrainLoss: 0.69146 | TestLoss: 0.69407 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 07 |  TrainLoss: 0.69201 | TestLoss: 0.69410 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 08 |  TrainLoss: 0.69262 | TestLoss: 0.69417 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 09 |  TrainLoss: 0.69138 | TestLoss: 0.69419 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 10 |  TrainLoss: 0.69220 | TestLoss: 0.69419 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 11 |  TrainLoss: 0.69057 | TestLoss: 0.69416 | 

# Model 3: 
GraphSage + Content with 2 Layer MLP as defined by the paper. (This is the model implemented in the paper)

In [17]:
import torch
import torch.nn as nn

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, embedding_size, batch_size, l2_reg_weight):
        super(Net, self).__init__()
        self.embedding_size = embedding_size
        self.batch_size = batch_size
        self.l2_reg_weight = l2_reg_weight
        
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        
        self.full1 = Linear(hidden_channels[2], hidden_channels[3])
        self.full2 = Linear(hidden_channels[3], hidden_channels[4])
        self.softmax = Linear(hidden_channels[4], out_channels)

        # Dropouts
        self.dp1 = Dropout(0.2)
        self.dp2 = Dropout(0.2)

    def forward(self, x, edge_index, batch):
        h = self.conv1(x, edge_index).relu()
        h = self.conv2(h, edge_index).relu()
        h = self.conv3(h, edge_index).relu()

        h = global_max_pool(h, batch)

        h = self.full1(h).relu()
        h = self.dp1(h)
        h = self.full2(h).relu()
        h = self.dp2(h)
        
        h = self.softmax(h)

        return torch.sigmoid(h)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(test_data_pol.num_features,[512,512,512,256,256,256],1, 128, 128, 0.001).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
lossff = torch.nn.BCELoss()
print(device)


cpu


In [19]:
def train(epoch):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        # print(out)

        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test(epoch):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    # print(preds.cpu().numpy())

    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1

In [20]:
wloss = []
weighted_loss = 0
exp_param = 0.8
best_test_loss = float('inf')  # Initialize with a large value

# Without dropout training results
for epoch in range(500):
  train_loss = train(epoch)
  test_loss, test_acc, test_f1 = test(epoch)
  weighted_loss = exp_param * (weighted_loss) + (1 - exp_param) * (test_loss / len(test_loader.dataset))
  
  wloss.append(weighted_loss / (1 - exp_param ** (epoch + 1)))
  
  if test_loss < best_test_loss:
    best_test_loss = test_loss  # Update the best test loss

  print(f'Epoch: {epoch:02d} |  TrainLoss: {train_loss:.5f} | '
        f'TestLoss: {test_loss:.5f} | TestAcc: {test_acc:.5f} | TestF1: {test_f1:.2f}')

# Print the best values
best_wloss = min(wloss)
best_epoch = wloss.index(best_wloss)
print(f'Best WLoss: {best_wloss:.5f} | Best Epoch: {best_epoch}')

Epoch: 00 |  TrainLoss: 0.69671 | TestLoss: 0.69282 | TestAcc: 0.51131 | TestF1: 0.68
Epoch: 01 |  TrainLoss: 0.69413 | TestLoss: 0.69280 | TestAcc: 0.51131 | TestF1: 0.68
Epoch: 02 |  TrainLoss: 0.69507 | TestLoss: 0.69285 | TestAcc: 0.57014 | TestF1: 0.69
Epoch: 03 |  TrainLoss: 0.69253 | TestLoss: 0.69307 | TestAcc: 0.49774 | TestF1: 0.03
Epoch: 04 |  TrainLoss: 0.69297 | TestLoss: 0.69333 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 05 |  TrainLoss: 0.68861 | TestLoss: 0.69350 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 06 |  TrainLoss: 0.68912 | TestLoss: 0.69365 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 07 |  TrainLoss: 0.68609 | TestLoss: 0.69376 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 08 |  TrainLoss: 0.68814 | TestLoss: 0.69375 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 09 |  TrainLoss: 0.69032 | TestLoss: 0.69369 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 10 |  TrainLoss: 0.69106 | TestLoss: 0.69328 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 11 |  TrainLoss: 0.68663 | TestLoss: 0.69253 | 

# Code 4: 
GraphSage + Content with hyperparameters as defined in the paper and replacemenent of 1 MLP layer with 1 RNN Layer:

In [41]:
import torch.nn as nn

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, embedding_size, batch_size, l2_reg_weight):
        super(Net, self).__init__()
        self.embedding_size = embedding_size
        self.batch_size = batch_size
        self.l2_reg_weight = l2_reg_weight
        
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        
        self.rnn = nn.RNN(embedding_size, hidden_channels[3], batch_first=True)
        self.full1 = nn.Linear(hidden_channels[3], hidden_channels[4])
        self.softmax = nn.Linear(hidden_channels[4], out_channels)

        # Dropouts
        self.dp1 = nn.Dropout(0.2)

    def forward(self, x, edge_index, batch):
        h = self.conv1(x, edge_index).relu()
        h = self.conv2(h, edge_index).relu()
        h = self.conv3(h, edge_index).relu()

        h = global_max_pool(h, batch)
        
        # Reshape the input tensor for RNN
        h = h.unsqueeze(0)  # Add a time dimension
        h = self.dp1(h)
        
        # Apply RNN
        h, _ = self.rnn(h)
        h = h.squeeze(0)  # Remove the time dimension
        
        h = self.full1(h).relu()
        h = self.softmax(h)

        return torch.sigmoid(h)

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(test_data_pol.num_features, [512, 512, 512, 256, 256, 256], 1, 512, 128, 0.001).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.BCELoss()
print(device)


wloss = []
weighted_loss = 0
exp_param = 0.8
best_test_loss = float('inf')  # Initialize with a large value

# Without dropout training results
for epoch in range(500):
    train_loss = train(epoch)
    test_loss, test_acc, test_f1 = test(epoch)
    weighted_loss = exp_param * weighted_loss + (1 - exp_param) * (test_loss / len(test_loader.dataset))
  
    wloss.append(weighted_loss / (1 - exp_param ** (epoch + 1)))
  
    if test_loss < best_test_loss:
        best_test_loss = test_loss  # Update the best test loss

    print(f'Epoch: {epoch:02d} | TrainLoss: {train_loss:.5f} | '
          f'TestLoss: {test_loss:.5f} | TestAcc: {test_acc:.5f} | TestF1: {test_f1:.2f}')

# Print the best values
best_wloss = min(wloss)
best_epoch = wloss.index(best_wloss)
print(f'Best WLoss: {best_wloss:.5f} | Best Epoch: {best_epoch}')

cpu
Epoch: 00 | TrainLoss: 0.69532 | TestLoss: 0.69257 | TestAcc: 0.51131 | TestF1: 0.21
Epoch: 01 | TrainLoss: 0.69383 | TestLoss: 0.69354 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 02 | TrainLoss: 0.69133 | TestLoss: 0.69481 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 03 | TrainLoss: 0.69154 | TestLoss: 0.69585 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 04 | TrainLoss: 0.69093 | TestLoss: 0.69642 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 05 | TrainLoss: 0.69183 | TestLoss: 0.69668 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 06 | TrainLoss: 0.69188 | TestLoss: 0.69624 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 07 | TrainLoss: 0.69076 | TestLoss: 0.69543 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 08 | TrainLoss: 0.68994 | TestLoss: 0.69437 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 09 | TrainLoss: 0.69015 | TestLoss: 0.69334 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 10 | TrainLoss: 0.68911 | TestLoss: 0.69225 | TestAcc: 0.48869 | TestF1: 0.00
Epoch: 11 | TrainLoss: 0.68867 | TestLoss: 0.69133 | TestAcc: